In [1]:
import numpy as np
import pandas as pd
import scipy.optimize as spop

4. <br>

The program of finding optimal dynamic strategy that $\displaystyle \max_{\theta}\mathbb{E}[u(W_T^{\theta})]$ with no trading costs, is implemented below.

Something that needs to being specified:

(1) When backward computing value function, suppose that $\displaystyle V_{k, j}$ is the value function for time $k$ and state $j$, and $\displaystyle \theta_{k, j}$ is the strategy for time $k$ and state $j$:

- Define gain function $\displaystyle g_k(\theta_k, \Delta \theta_k) = u(\theta_k)$, which does not depend on $ \Delta \theta_k$

- At time $k$ and state $j_0$, define value function $$\displaystyle \bar{V_{k, j_0}}(\theta_{k, j_0}, \Delta \theta_{k, j_0}) = g_k(\theta_{k, j_0}, \Delta \theta_{k, j_0}) + \sum_{j = 1}^{nS} [(V_{k+1, j}(\theta_{k, j_0} + \Delta \theta_{k, j_0})\text{Pr}(\text{State } j_0 \text{ to State } j))] = u(\theta_{k, j_0}) + \sum_{j = 1}^{nS}[\cdots]$$ where $nS$ is the number of state. <br>
- This is because different states at time $k + 1$ has different value functions, so we take their mean to compute value function at time $k$. <br>
At time $T$, we can set $\displaystyle V_{T+1, j}(\theta_{T, j_0} + \Delta \theta_{T, j_0}) = 0, \ \forall \, j, j_0$ <br>

- Then our objective at time $k$ and state $j_0$ is: $$\displaystyle \max_{\Delta \theta_{k, j_0}}\bar{V_{k, j_0}}(\theta_{k, j_0}, \Delta \theta_{k, j_0})$$<br>

- Note that there is no trading costs, so $$V(\theta_{k, j_0}) = \displaystyle \max_{\Delta \theta_{k, j_0}}\bar{V_{k, j_0}}(\theta_{k, j_0}, \Delta \theta_{k, j_0}) \equiv \bar{V_{k, j_0}}(\theta_{k, j_0}, \cdot)$$

- Since our objective does not depends on $\Delta \theta_{k, j_0}$, subsequently does not depend on $\theta_{k+1, j}$ and $\theta_{k-1, j}, \ \forall \, j$.

- Hence we can directly choose the optimal $\displaystyle \theta_{k, j_0}$ that maximize $\displaystyle V(\theta_{k, j_0})$ at time $k$,  $\displaystyle \forall \, k = 1, 2, \ldots, T$, and these choices form the dynamic optimal strategy. We do not need the forward step to comupte optimal $\theta$.

- All $\theta_{k, j}$ are represented in dollar unit, so we compute security's price at state $j$ as $p_j$, and set the number of shares in our strategy: $\displaystyle \frac{\theta_{k, j}}{p_j}$ 

In [2]:
# Question 4, optimal strategy with no trading costs

def dpStrategy(S, trS, T, p, u):
    
    '''
    the program that finds self-financing strategy of maximizing utility expectation with no trading costs
    Parameters:
       S: the finite state space, suppose the number of state is nS
       trS: the transition matrix of the finite state space
       T: number of states in this strategy, X1, X2,..., XT
       p: the price function of the Markov Chain
       u: the investor's utility function 
    Return: 
       M: an (nS*T) matrix that represents the optimal strategy at each time Xt, t = 1,2,...T
          where the k-th column represents the optimal strategy at each state at Xk
    '''
    
    # given the state space and transition matrix
    # compute mean and variance for investment at each state
    nS = len(S)
    # initialize arrays to record mean and standard deviation
    muS = np.zeros(nS)
    sigmaS = np.zeros(nS)
    for i in range(nS):
        nextSprob = trS[i, :]
        mu = nextSprob.dot(S.reshape(nS, 1)) # mean
        sigma = 0
        for j in range(nS):
            sigma += nextSprob[0,j] * (S[j] - mu)**2 # variance
        muS[i], sigmaS[i] = mu, np.sqrt(sigma) # record mean and standard deviation
    
    # initialize the matrix of value function
    V = np.zeros((nS, T + 1))
    # initialize the matrix of result
    M = np.zeros((nS, T + 1))
    # compute optimal strategy at each time step and each state
    # detailed algorithm is showed above
    for i in range(T, -1, -1):
        for j in range(nS):
            # construct the objective function that include utility function and next time's value function
            # take the negative so python built-in minimization can be used
            if (i == T):
                uNeg = lambda x, mu, sigma: -(u(x, mu, sigma)) 
            else:
                # next time's value function contains each state's value function at next time
                uNeg = lambda x, mu, sigma: -(u(x, mu, sigma) + V[:, i+1].dot(S.reshape(nS, 1))) 
            # compute the optimal amount given current state
            optimalAmount = spop.minimize(uNeg, 0, (muS[j], sigmaS[j])).x
            # compute the current price of the security for self-financing purpose
            # and to compute optimal shares
            price = p(S[j])
            # record the result
            M[j][i] = optimalAmount / price
            if (i == T):
                V[j, i] = u(optimalAmount, muS[j], sigmaS[j])
            else:
                V[j, i] = u(optimalAmount, muS[j], sigmaS[j]) + V[:, i+1].dot(S.reshape(nS, 1))
    return pd.DataFrame(M, columns = ['Time %d' % i for i in range(T + 1)], 
                        index = ['State %d' % i for i in range(1, nS + 1)])

In [3]:
# define the finite state space set in all exapmles
# each state represents the PnL per dollar investment
S_def = np.array([-0.02, -0.01, 0, 0.015, 0.025])
# define a transition matrix for the state space
trS_def = np.matrix([[0.35, 0.25, 0.2, 0.15, 0.05], 
                 [0.3, 0.25, 0.2, 0.15, 0.1], 
                 [0.2, 0.2, 0.2, 0.2, 0.2], 
                 [0.1, 0.15, 0.2, 0.25, 0.3], 
                 [0.05, 0.15, 0.2, 0.25, 0.35]])
# define the number of time step in all examples
T_def = 3

# default investor's utility function, u(x) = mu*x - (sigma*x)^2/2
# since this function is equivalent to the expectation of exponential utility
# this function is applied to find the optimal strategy in all exapmles
u_def = lambda x, mu, sigma: mu * x - (sigma * x)**2 / 2

# default price function of the Markov Chain
# this function is applied to find the optimal strategy in all exapmles
p_def = lambda x: 1 + x

# display the result
print("The optimal dynamic strategy(in shares) for example defined above is:")
print("")
print(dpStrategy(S = S_def, trS = trS_def, T = T_def, p = p_def, u = u_def))

The optimal dynamic strategy(in shares) for example defined above is:

            Time 0     Time 1     Time 2     Time 3
State 1 -31.559010 -31.559013 -31.559013 -31.559002
State 2 -16.672919 -16.672922 -16.672915 -16.672919
State 3   7.518796   7.518796   7.518796   7.518796
State 4  31.989398  31.989398  31.989398  31.989386
State 5  46.457613  46.457613  46.457613  46.457630


An example of applying the program to find dynamic strategy of the objective $\displaystyle \max_{\theta}\mathbb{E}[u(W_T^{\theta})]$ is showed above. 

In this example, I set the utility function as $\displaystyle u(x) = \mu x - \frac{\sigma^2 x^2}{2}$.

This is because for exponential utility function, if $\displaystyle v(w) = -e^{-w}$, then $\displaystyle \mathbb{E}[v(w)] \iff \mu_w w - \frac{\sigma_w^2 w^2}{2}$

5. <br>

The program of finding optimal dynamic strategy with Garleanu-Pedersen-type objective is implemented below.

The first function is considering linear trading costs, and the second is considering quadratic trading costs.

Something that needs to being specified:

(1) When backward computing value function, suppose that $\displaystyle V_{k, j}$ is the value function for time $k$ and state $j$, and $\displaystyle \theta_{k, j}$ is the strategy for time $k$ and state $j$:

- Define gain function $\displaystyle g_k(\theta_k, \Delta \theta_k) = \mu\theta_k - \frac{\sigma^2\theta_k^2}{2} - \lambda\vert\Delta \theta_k\vert$ for linear trading costs. ($\displaystyle \frac{\lambda(\Delta \theta_k)^2}{2}$ for quadratic) <br>

- At time $k$ and state $j_0$, define value function $$\displaystyle \bar{V_{k, j_0}}(\theta_{k, j_0}, \Delta \theta_{k, j_0}) = g_k(\theta_{k, j_0}, \Delta \theta_{k, j_0}) + \sum_{j = 1}^{nS} [(V_{k+1, j}(\theta_{k, j_0} + \Delta \theta_{k, j_0})\text{Pr}(\text{State } j_0 \text{ to State } j))]$$ where $nS$ is the number of state. <br>
- This is because different states at time $k + 1$ has different value functions, so we take their mean to compute value function at time $k$. <br>
At time $T$, we can set $\displaystyle V_{T+1, j}(\theta_{T, j_0} + \Delta \theta_{T, j_0}) = 0, \ \forall \, j, j_0$ <br>

- Then our objective at time $k$ and state $j_0$ is: $$\displaystyle \max_{\Delta \theta_{k, j_0}}\bar{V_{k, j_0}}(\theta_{k, j_0}, \Delta \theta_{k, j_0})$$<br>

- By taking partial derivatives and set to zero: $\displaystyle \partial_{\Delta \theta_{k, j_0}}\bar{V_{k, j_0}}(\theta_{k, j_0}, \Delta \theta_{k, j_0}) = 0$, we can have a relationship between $\theta_{k, j_0}$ and $\Delta \theta_{k, j_0}: \Delta \theta_{k, j_0} = d_{k, j_0}(\theta_{k, j_0})$<br>

- Replace $\Delta \theta_{k, j_0}$ by $d_{k, j_0}(\theta_{k, j_0})$ in $\displaystyle \bar{V_{k, j_0}}(\theta_{k, j_0}, \Delta \theta_{k, j_0})$, <br> 
we can have that: $$V_{k, j_0}(\theta_{k, j_0}) = \max_{\Delta \theta_{k, j_0}}\bar{V_{k, j_0}}(\theta_{k, j_0}, \Delta \theta_{k, j_0}) = \bar{V_{k, j_0}}(\theta_{k, j_0}, d_{k, j_0}(\theta_{k, j_0}))$$

which is a value function only depends on $\theta_{k, j_0}$.<br>

(2) After computing all value functions, $V_{0, j}(\theta_{0, j}), V_{1, j}(\theta_{1, j}), \cdots$, we compute optimal strategy forward recursively.

- At time $0$, find $\theta_{0, j}$ that maximizes $V_{0, j}(\theta_{0, j}), \forall \, j$

- At time $k$, we have known the strategy at time $k - 1$ and the relationship between $\theta_{k-1, j}$ and $\Delta \theta_{k-1, j}$: $\displaystyle \Delta \theta_{k-1, j} = d_{k-1, j}(\theta_{k-1, j}), \forall \, j$

- Then at time $k$ and state $j_0$: $$\theta_{k, j_0} = \sum_{j = 1}^{nS} [(\theta_{k - 1, j} + d_{k-1, j}(\theta_{k-1, j}))\text{Pr}(\text{State } j \text{ to State } j_0)]$$ since different states at time $k - 1$ has different optimal strategies, so we take their mean

- All $\theta_{k, j}$ are represented in dollar unit, so we compute security's price at state $j$ as $p_j$, and set the number of shares in our strategy: $\displaystyle \frac{\theta_{k, j}}{p_j}$ 

In [4]:
# Question 5(a), optimal strategy with linear trading costs

import sympy as sp

def dpStrategy_lin(S, trS, T, lbd, p):
    
    '''
    the program that finds self-financing strategy of maximizing Garleanu-Pedersen-type
        objective with linear trading costs
    Parameters:
       S: the finite state space, suppose the number of state is nS
       trS: the transition matrix of the finite state space
       T: number of states in this strategy, X1, X2,..., XT
       lbd: the parameter of linear trading costs
       p: the price function of the Markov Chain
    Return: 
       M: an (nS*T) matrix that represents the optimal strategy at each time Xt, t = 1,2,...T
          where the k-th column represents the optimal strategy at each state at Xk
    '''
    # given the state space and transition matrix
    # compute mean and variance for investment at each state
    nS = len(S)
    # initialize arrays to record mean and standard deviation
    muS = np.zeros(nS)
    sigmaS = np.zeros(nS)
    for i in range(nS):
        nextSprob = trS[i, :]
        mu = nextSprob.dot(S.reshape(nS, 1)) # mean
        sigma = 0
        for j in range(nS):
            sigma += nextSprob[0,j] * (S[j] - mu)**2 # variance
        muS[i], sigmaS[i] = mu, np.sqrt(sigma) # record mean and standard deviation
        
    # initialize the matrix of value function
    V = [[0] * (T + 1) for i in range(nS)]
    # initialize the matrix to store the relationship between theta_k and (delta theta_k) for k = 0, ..., T 
    rel = [[0] * (T + 1) for i in range(nS)]
    # detailed algorithm for finding value function and relationship can be found above
    for i in range(T, -1, -1):
        for j in range(nS):
            # update GPtype objective function for time Ti by value function of T_(i + 1)
            x, y = sp.symbols('x,y',real=True)
            if (i == T):
                # construct V_hat(theta, (delta theta))
                obj_ori = muS[j] * x - (sigmaS[j] * x)**2 / 2 - lbd*y
                for k in range(nS):
                    obj_ori += (muS[k]*(x+y) - (sigmaS[k]*(x+y))**2/2) * trS[j, k]
                # find relationship between theta and (delta theta))
                obj_rel = sp.solve(obj_ori.diff(y), y)
                # record the result
                rel[j][i] = obj_rel[0]
                # construct V(theta) by V_hat(theta, (delta theta)) and relationship between theta and (delta theta))
                obj_new = muS[j] * x - (sigmaS[j] * x)**2 / 2 - lbd*(obj_rel[0])
                for k in range(nS):
                    obj_new += (muS[k]*(x+obj_rel[0]) - (sigmaS[k]*(x+obj_rel[0]))**2/2) * trS[j, k]
                # record the result
                V[j][i] = obj_new
            else:
                # construct V_hat(theta, (delta theta))
                obj_ori = muS[j] * x - (sigmaS[j] * x)**2 / 2 - lbd*y
                for k in range(nS):
                    obj_ori += V[k][i+1].replace(x, x+y) * trS[j, k]
                # find relationship between theta and (delta theta))
                obj_rel = sp.solve(obj_ori.diff(y), y)
                # record the result
                rel[j][i] = obj_rel[0]
                # construct V(theta) by V_hat(theta, (delta theta)) and relationship between theta and (delta theta))
                obj_new = muS[j] * x - (sigmaS[j] * x)**2 / 2 - lbd*(obj_rel[0])
                for k in range(nS):
                    obj_new += V[k][i+1].replace(x, x+obj_rel[0]) * trS[j, k]
                # record the result
                V[j][i] = obj_new

    # initialize the matrix of result
    M = np.zeros((nS, T + 1))
    # detailed algorithm for optimal strategy can be found above
    for i in range(T + 1):
        for j in range(nS):
            if (i == 0):
                # compute the current price of the security for self-financing purpose
                # and to compute optimal shares
                price = p(S[j])
                # compute optimal strategy at time 0
                # take the negative of value function so python built-in minimization can be used
                M[j, i] = spop.minimize(lambda xs: -sp.lambdify(x, V[j][i])(xs), 0).x / price 
            else:
                price = p(S[j])
                # compute optimal strategy at time k given optimal strategy at time (k - 1)
                sum = 0
                for k in range(nS):
                    sum += (M[k, i - 1] + sp.lambdify(x, V[k][i - 1])(M[k, i - 1])) * trS[j, k]
                M[j, i] = (sum / price)

    return pd.DataFrame(M, columns = ['Time %d' % i for i in range(T + 1)], 
                        index = ['State %d' % i for i in range(1, nS + 1)])

In [5]:
# Question 5(b), optimal strategy with quadratic trading costs

def dpStrategy_quad(S, trS, T, lbd, p):
    
    '''
    the program that finds self-financing strategy of maximizing Garleanu-Pedersen-type
        objective with quadratic trading costs
    Parameters:
       S: the finite state space, suppose the number of state is nS
       trS: the transition matrix of the finite state space
       T: number of states in this strategy, X1, X2,..., XT
       lbd: the parameter of quadratic trading costs
       p: the price function of the Markov Chain
    Return: 
       M: an (nS*T) matrix that represents the optimal strategy at each time Xt, t = 1,2,...T
          where the k-th column represents the optimal strategy at each state at Xk
    '''
    # given the state space and transition matrix
    # compute mean and variance for investment at each state
    nS = len(S)
    # initialize arrays to record mean and standard deviation
    muS = np.zeros(nS)
    sigmaS = np.zeros(nS)
    for i in range(nS):
        nextSprob = trS[i, :]
        mu = nextSprob.dot(S.reshape(nS, 1)) # mean
        sigma = 0
        for j in range(nS):
            sigma += nextSprob[0,j] * (S[j] - mu)**2 # variance
        muS[i], sigmaS[i] = mu, np.sqrt(sigma) # record mean and standard deviation
        
    # initialize the matrix of value function
    V = [[0] * (T + 1) for i in range(nS)]
    # initialize the matrix to store the relationship between theta_k and (delta theta_k) for k = 0, ..., T 
    rel = [[0] * (T + 1) for i in range(nS)]
    # detailed algorithm for finding value function and relationship can be found above
    for i in range(T, -1, -1):
        for j in range(nS):
            # update GPtype objective function for time Ti by value function of T_(i + 1)
            x, y = sp.symbols('x,y',real=True)
            if (i == T):
                # construct V_hat(theta, (delta theta))
                obj_ori = muS[j] * x - (sigmaS[j] * x)**2 / 2 - lbd*y**2/2
                for k in range(nS):
                    obj_ori += (muS[k]*(x+y) - (sigmaS[k]*(x+y))**2/2) * trS[j, k]
                # find relationship between theta and (delta theta))
                obj_rel = sp.solve(obj_ori.diff(y), y)
                # record the result
                rel[j][i] = obj_rel[0]
                # construct V(theta) by V_hat(theta, (delta theta)) and relationship between theta and (delta theta))
                obj_new = muS[j] * x - (sigmaS[j] * x)**2 / 2 - lbd*(obj_rel[0])**2/2
                for k in range(nS):
                    obj_new += (muS[k]*(x+obj_rel[0]) - (sigmaS[k]*(x+obj_rel[0]))**2/2) * trS[j, k]
                # record the result
                V[j][i] = obj_new
            else:
                # construct V_hat(theta, (delta theta))
                obj_ori = muS[j] * x - (sigmaS[j] * x)**2 / 2 - lbd*y**2/2
                for k in range(nS):
                    obj_ori += V[k][i+1].replace(x, x+y) * trS[j, k]
                # find relationship between theta and (delta theta))
                obj_rel = sp.solve(obj_ori.diff(y), y)
                # record the result
                rel[j][i] = obj_rel[0]
                # construct V(theta) by V_hat(theta, (delta theta)) and relationship between theta and (delta theta))
                obj_new = muS[j] * x - (sigmaS[j] * x)**2 / 2 - lbd*(obj_rel[0])**2/2
                for k in range(nS):
                    obj_new += V[k][i+1].replace(x, x+obj_rel[0]) * trS[j, k]
                # record the result
                V[j][i] = obj_new

    # initialize the matrix of result
    M = np.zeros((nS, T + 1))
    # detailed algorithm for optimal strategy can be found above
    for i in range(T + 1):
        for j in range(nS):
            if (i == 0):
                # compute the current price of the security for self-financing purpose
                # and to compute optimal shares
                price = p(S[j])
                # compute optimal strategy at time 0
                # take the negative of value function so python built-in minimization can be used
                M[j, i] = spop.minimize(lambda xs: -sp.lambdify(x, V[j][i])(xs), 0).x / price 
            else:
                price = p(S[j])
                # compute optimal strategy at time k given optimal strategy at time (k - 1)
                sum = 0
                for k in range(nS):
                    sum += (M[k, i - 1] + sp.lambdify(x, V[k][i - 1])(M[k, i - 1])) * trS[j, k]
                M[j, i] = (sum / price)

    return pd.DataFrame(M, columns = ['Time %d' % i for i in range(T + 1)], 
                        index = ['State %d' % i for i in range(1, nS + 1)])

In [6]:
# display the result for linear trading costs
print("The optimal dynamic strategy(in shares) considering linear trading costs is:")
print("")
print(dpStrategy_lin(S = S_def, trS = trS_def, T = T_def, lbd = 0.005, p = p_def))

print("")
print("")

# display the result for quadratic trading costs
print("The optimal dynamic strategy(in shares) considering quadratic trading costs is:")
print("")
print(dpStrategy_quad(S = S_def, trS = trS_def, T = T_def, lbd = 1, p = p_def))

The optimal dynamic strategy(in shares) considering linear trading costs is:

            Time 0     Time 1     Time 2     Time 3
State 1  -5.259833  16.702092  25.637264  29.027332
State 2   5.557649  20.349195  26.719865  29.191068
State 3  26.315805  30.074074  30.081199  30.146273
State 4  52.627676  39.411277  33.211558  30.929446
State 5  69.686388  42.712291  34.183296  31.069034


The optimal dynamic strategy(in shares) considering quadratic trading costs is:

            Time 0     Time 1    Time 2    Time 3
State 1  -2.477055   4.604484  7.284988  8.288116
State 2   0.747211   5.688636  7.620971  8.348112
State 3   8.512394   8.661306  8.654959  8.656687
State 4  16.405064  11.518091  9.620844  8.915017
State 5  19.684974  12.497773  9.922566  8.966848


The example of applying the program to find optimal dynamic strategy of Garleanu-Pedersen-type objective, with linear and quadratic trading costs respectively, is showed above.